In [1]:
#!pip install spacy
#!python -m spacy download en_core_web_sm
import spacy
from spacy.matcher import Matcher
from zipfile import ZipFile
from pathlib import Path
from tqdm import autonotebook as tqdm

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
with Path("./data/teaching-dataset/sentence_classification_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = f.read().decode("utf-8").splitlines()
with Path("./data/teaching-dataset/sentence_classification_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = f.read().decode("utf-8").splitlines()

In [4]:
for label, sentence in zip(labels[:10], sentences[:10]):
    print(label, sentence)

1 Herbal medications, spider bites, iopamidol (used for radiocontrast), lacquers, mercury, psoralen (combined with ultraviolet A to treat psoriasis), and xenobiotics have been associated with the development of AGEP in case reports.
1 The jar from the fall had caused a subdural hematoma on the right front side of my brain that shows up in the CT.
1 It's very rewarding and I get a great deal of enjoyment and satisfaction from mentoring.
1 The video then shows the damage caused by the aircraft as it hit the north tower, follows the disintegrating plane through the interior, and then shows the airplane metal, ignited fuel, dust and smoke exiting the building on the opposite side.
0 It has been assumed that the quench starts at the middle cross-section of the magnet.
0 Her diamonds are locked in a safe deposit box.
0 A large marble was dropped into the bowl.
1 Relatives of the prisoners gathered outside the facility to discover the fate of the incarcerated, eventually leading to clashes wi

In [5]:
#count most common words in sentences that have causal relationship
from collections import Counter
from collections.abc import Iterable

# from: https://stackoverflow.com/questions/17485747/how-to-convert-a-nested-list-into-a-one-dimensional-list-in-python
def flatten(lis): #pretty ugly solution but we have to flatten the list since every new sentence adds "[]" which Counter can't deal with
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

                
words = []

for label, sentence in zip(labels, sentences):
    if label == "1": #if sentence is causal
        doc = nlp(sentence)
        wordsHelp = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct and token.pos_ != "NOUN"] #append all words to a list if they are NOT nouns & not punctuation
        words.append(wordsHelp)
        
# five most common tokens
word_freq = Counter(flatten(words))
common_words = word_freq.most_common(20)
print(common_words)

[('caused', 486), ('cause', 110), ('associated', 42), ('resulted', 40), ('causes', 40), ('common', 38), ('triggered', 37), ('causing', 33), ('comes', 32), ('lead', 32), ('include', 31), ('leading', 29), ('increased', 29), ('high', 28), ('severe', 27), ('found', 25), ('like', 24), ('large', 24), ('generated', 23), ('produced', 22)]


In [6]:
doc = nlp(sentences[0])
for token in doc:
    print(token.text, token.pos_, token.dep_)

Herbal ADJ amod
medications NOUN nsubjpass
, PUNCT punct
spider NOUN compound
bites NOUN appos
, PUNCT punct
iopamidol PROPN conj
( PUNCT punct
used VERB parataxis
for ADP prep
radiocontrast NOUN pobj
) PUNCT punct
, PUNCT punct
lacquers NOUN nsubjpass
, PUNCT punct
mercury NOUN nmod
, PUNCT punct
psoralen PROPN conj
( PUNCT punct
combined VERB prep
with ADP prep
ultraviolet ADJ compound
A NOUN pobj
to PART aux
treat VERB advcl
psoriasis NOUN dobj
) PUNCT punct
, PUNCT punct
and CCONJ cc
xenobiotics NOUN conj
have AUX aux
been AUX auxpass
associated VERB ROOT
with ADP prep
the DET det
development NOUN pobj
of ADP prep
AGEP PROPN pobj
in ADP prep
case NOUN compound
reports VERB pobj
. PUNCT punct


In [7]:
causal_cues = ["cause", "because", "since", "due to", "causes"]

for words, counts in common_words:
    causal_cues.append(str(words))

# Naive Improvements
- co-occurence frequencies
- patterns in syntactic / dependency trees
- causal verbs / causal phrases
- lemmatization

In [8]:
# def predict(sentence, causal_cues):
#     doc = nlp(sentence)
#     for token in doc:
#         if token.text.lower() in causal_cues:
#             return 1
#     return 0

def predict(sentence, causal_cues):
    # matcher can be found at https://spacy.io/api/matcher
    matcher = Matcher(nlp.vocab)
    pattern = [[{"LEMMA": cue}] for cue in causal_cues]
    matcher.add("CAUSAL", pattern)

    doc = nlp(sentence)
    matches = matcher(doc)
    return bool(matches)

In [9]:
predictions = []
for sentence in tqdm.tqdm(sentences):
    predictions.append(predict(sentence, causal_cues))

  0%|          | 0/3260 [00:00<?, ?it/s]

In [10]:
tp = sum([int(p) == 1 and int(l) == 1 for p, l in zip(predictions, labels)])
fp = sum([int(p) == 1 and int(l) == 0 for p, l in zip(predictions, labels)])
tn = sum([int(p) == 0 and int(l) == 0 for p, l in zip(predictions, labels)])
fn = sum([int(p) == 0 and int(l) == 1 for p, l in zip(predictions, labels)])
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")

Precision: 0.87
Recall: 0.57
F1: 0.69
Accuracy: 0.74


In [11]:
with Path("./data/teaching-dataset/sentence_classification_text_test.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        test_sentences = f.read().decode("utf-8").splitlines()

In [12]:
test_predictions = []
for sentence in tqdm.tqdm(test_sentences):
    test_predictions.append(predict(sentence, causal_cues))
Path("predictions.txt").write_text("\n".join(map(str, test_predictions)));

  0%|          | 0/840 [00:00<?, ?it/s]